D'après une étude VIF, il faudrait supprimer les colonnes ZipCode, Latitude, Longitude

In [ ]:
import pandas as pd
import seaborn as sns
import missingno as msno
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime

from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import OneHotEncoder

from sklearn import model_selection

In [11]:
df = pd.read_csv("data_cleaned.csv")
df.drop(['ZipCode','Latitude', 'Longitude'], axis=1, inplace=True)

In [12]:
X = df.drop('TotalGHGEmissions', axis=1)
y = df['TotalGHGEmissions']

In [13]:
X_train, X_test, Y_train, Y_test = model_selection.train_test_split(X, y,test_size=0.3)

In [14]:
X_train.describe()

,CouncilDistrictCode,NumberofFloors,PropertyGFATotal,PropertyGFAParking,ENERGYSTARScore,SteamUse(kBtu),BuildingAge
count,1147.000000,1147.000000,1.147000e+03,1147.000000,755.000000,1.147000e+03,1147.000000
mean,4.432432,4.147341,1.218090e+05,13155.698344,64.919205,3.760828e+05,53.859634
std,2.201759,6.619429,3.309629e+05,41840.698685,28.410081,3.180311e+06,32.780357
min,1.000000,0.000000,1.128500e+04,0.000000,1.000000,0.000000e+00,1.000000
25%,2.000000,1.000000,3.000000e+04,0.000000,47.000000,0.000000e+00,27.000000
50%,4.000000,2.000000,4.974400e+04,0.000000,72.000000,0.000000e+00,51.000000
75%,7.000000,4.000000,1.100930e+05,0.000000,89.000000,0.000000e+00,86.000000
max,7.000000,99.000000,9.320156e+06,512608.000000,100.000000,8.498524e+07,116.000000


In [7]:
imputer = KNNImputer(n_neighbors=5)
imputer = imputer.fit(X_train[["ENERGYSTARScore"]])
X_train[["ENERGYSTARScore"]] = imputer.transform(X_train[["ENERGYSTARScore"]])
X_test[["ENERGYSTARScore"]] = imputer.transform(X_test[["ENERGYSTARScore"]])

In [8]:
# RobustScaler
rs = RobustScaler()
rs = rs.fit(X_train[['CouncilDistrictCode', 'NumberofFloors', 'PropertyGFATotal', 'PropertyGFAParking', 'ENERGYSTARScore', 'SteamUse(kBtu)', 'BuildingAge']])
X_train[['CouncilDistrictCode', 'NumberofFloors', 'PropertyGFATotal', 'PropertyGFAParking', 'ENERGYSTARScore', 'SteamUse(kBtu)', 'BuildingAge']] = rs.transform(X_train[['CouncilDistrictCode', 'NumberofFloors', 'PropertyGFATotal', 'PropertyGFAParking', 'ENERGYSTARScore', 'SteamUse(kBtu)', 'BuildingAge']])
X_test[['CouncilDistrictCode', 'NumberofFloors', 'PropertyGFATotal', 'PropertyGFAParking', 'ENERGYSTARScore', 'SteamUse(kBtu)', 'BuildingAge']] = rs.transform(X_test[['CouncilDistrictCode', 'NumberofFloors', 'PropertyGFATotal', 'PropertyGFAParking', 'ENERGYSTARScore', 'SteamUse(kBtu)', 'BuildingAge']])

In [ ]:
# OneHotEncoder
ohe = OneHotEncoder(sparse=False)
ohe = ohe.fit(X_train[['BuildingType', 'PrimaryPropertyType', 'Neighborhood']])
# X_train[['BuildingType', 'PrimaryPropertyType', 'Neighborhood']] = ohe.transform(X_train[['BuildingType', 'PrimaryPropertyType', 'Neighborhood']])

In [ ]:
ohe_trained = pd.DataFrame(ohe.transform(X_train[['BuildingType', 'PrimaryPropertyType', 'Neighborhood']]), columns=ohe.get_feature_names())

In [ ]:
X_train = X_train.drop(columns={'BuildingType', 'PrimaryPropertyType', 'Neighborhood'}, axis = 1)
X_train = X_train.join(ohe_trained)

In [ ]:
X_train